In [1]:
import os
import re
import time


import requests
from parsel import Selector
import yt_dlp
from playwright.async_api import async_playwright

In [2]:
start = time.time()

In [3]:
URL = [
    "https://www.moissaniteco.com/moissanite/enr198-mq/marquise-moissanite-scroll-accented-solitaire-ring",
    "https://www.moissaniteco.com/moissanite/enr173-cu/cushion-moissanite-bezel-solitaire-ring",
    # "https://www.moissaniteco.com/moissanite/enr173-cu/cushion-moissanite-bezel-solitaire-ring",
    # "https://www.moissaniteco.com/moissanite/enr198-mq/marquise-moissanite-scroll-accented-solitaire-ring",
    # "https://www.moissaniteco.com/moissanite/enr603/round-moissanite-graduated-halo-engagement-ring",
    # "https://www.moissaniteco.com/moissanite/enr052-em/emeraldradiant-and-round-moissanite-3-stone-ring-with-scrolls",
    # So on
]

In [4]:
# Playwright code
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(
    channel="chrome",
    # headless=False,
    args=["--disable-blink-features", "--disable-blink-features=AutomationControlled"],
    # timeout=0
)
context = await browser.new_context(
    user_agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
)
page = await context.new_page()

In [5]:
count = 0
for PRODUCT_URL in URL:
    
    # Product url counting code
    count +=1
    print(f"===== product URL number {count} =======")
    

    # Palywright html content code
    await page.goto(PRODUCT_URL, wait_until="domcontentloaded")
    page.wait_for_timeout(5000)
    html = await page.content()
    response = Selector(text=html)
    print("===== playwright page content part ======")
    
    
    
    
    
    
    
    
    
    
    # Image and video downloader code
    main_page = response.xpath("//div[@class='cell cell--media']//div[@class='media-main-image grid-x align-middle']/div/img/@src").get()
    if main_page.startswith("https://www.moissaniteco.com/"):
        _5_images_url_list = response.xpath("//div[@class='thumb thumb--images']/a/@href").getall()
        hidden_images_url_list = response.xpath("//a[@data-fancybox='images' and @class='hide']/@href").getall()
        orginal_videos_url_list = response.xpath("//a[@data-fancybox='videos' and @class='hide']/@href").getall()
        youtube_videos_url_list = response.xpath("//a[@data-fancybox='youtube-videos' and @class='hide']/@href").getall()
        
        
        # Product folder creation code
        match = re.search(r"/moissanite/([\w-]+)/", PRODUCT_URL)
        if match:
            product_id = match.group(1).upper()
        directory_name = f"AD{product_id}"
        os.makedirs(directory_name, exist_ok=True)
        os.makedirs(os.path.join(directory_name, "images"), exist_ok=True)
        os.makedirs(os.path.join(directory_name, "Video"), exist_ok=True)
        os.makedirs(os.path.join(directory_name, "Real Video"), exist_ok=True)
        
        
        
        
        

        
        
        
        # main image downloader code
        main_page = response.xpath("//div[@class='cell cell--media']//div[@class='media-main-image grid-x align-middle']/div/img/@src").get()
        # main image downloader code
        match = re.search(r"/([\w-]+)\.jpg$", main_page)
        if match:
            image_id = match.group(1)
            headers = {
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
            }

            response = requests.get(main_page, headers=headers)
            image_file_name = f"{image_id}.jpg"

            # Save file in the images subdirectory of the product directory
            with open(os.path.join(directory_name, "images", image_file_name), 'wb') as file:
                file.write(response.content)



        # _5 images downloader code
        for url in _5_images_url_list:
            match = re.search(r"/([\w-]+)\.jpg$", url)
            if match:
                image_id = match.group(1)
                headers = {
                    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
                }

                response = requests.get(url, headers=headers)
                image_file_name = f"{image_id}.jpg"

                # Save file in the images subdirectory of the product directory
                with open(os.path.join(directory_name, "images", image_file_name), 'wb') as file:
                    file.write(response.content)
                    
                
        
        # hidden image downloader code
        for url in hidden_images_url_list:
            match = re.search(r"/([\w-]+)\.jpg$", url)
            if match:
                image_id = match.group(1)
                headers = {
                    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
                }

                response = requests.get(url, headers=headers)
                image_file_name = f"{image_id}.jpg"

                # Save file in the images subdirectory of the product directory
                with open(os.path.join(directory_name, "images", image_file_name), 'wb') as file:
                    file.write(response.content)
                    
        
        
        # Videos downloader code
        for url in orginal_videos_url_list:
            match = re.search(r"/([\w-]+)\.mp4$", url)
            if match:
                video_id = match.group(1)
                headers = {
                    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
                }

                response = requests.get(url, headers=headers)
                video_file_name = f"{video_id}.mp4"

                # Save file in the Real Video subdirectory of the product directory
                with open(os.path.join(directory_name, "Real Video", video_file_name), 'wb') as file:
                    file.write(response.content)
                    
        
        # YouTube Videos downloader code
        for url in youtube_videos_url_list:
            # Clean the URL
            clean_url = re.sub(r"\?.*$", "", url)

            # Extract the video ID from the URL
            match = re.search(r"/embed/([\w-]+)", clean_url)
            if match:
                video_id = match.group(1)

                ydl_opts = {
                    'outtmpl': os.path.join(directory_name, "Video", f"{video_id}.%(ext)s")
                }

                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    ydl.download([clean_url])
        
        
       
    
    
    
    
    
    
    
    # Error handling code
    else:
        print("*****************************")
        print(PRODUCT_URL)
        print("******************************")
        

    
    

    

===== product URL number 1 =======


/tmp/ipykernel_20207/4253731750.py:22: RuntimeWarning: coroutine 'Page.wait_for_timeout' was never awaited
  page.wait_for_timeout(5000)


===== playwright page content part ======
[youtube] Extracting URL: https://www.youtube.com/embed/F4vlm6T4-aM
[youtube] F4vlm6T4-aM: Downloading webpage
[youtube] F4vlm6T4-aM: Downloading ios player API JSON
[youtube] F4vlm6T4-aM: Downloading android player API JSON
[youtube] F4vlm6T4-aM: Downloading m3u8 information
[info] F4vlm6T4-aM: Downloading 1 format(s): 22
[download] Destination: ADENR198-MQ/Video/F4vlm6T4-aM.mp4
[download] 100% of    2.54MiB in 00:00:05 at 447.01KiB/s   
[youtube] Extracting URL: https://www.youtube.com/embed/f8Knlnh5GzI
[youtube] f8Knlnh5GzI: Downloading webpage
[youtube] f8Knlnh5GzI: Downloading ios player API JSON
[youtube] f8Knlnh5GzI: Downloading android player API JSON
[youtube] f8Knlnh5GzI: Downloading m3u8 information
[info] f8Knlnh5GzI: Downloading 1 format(s): 22
[download] Destination: ADENR198-MQ/Video/f8Knlnh5GzI.mp4
[download] 100% of    4.12MiB in 00:00:06 at 605.20KiB/s 
[youtube] Extracting URL: https://www.youtube.com/embed/6O2GyvS-uL0
[youtu

/tmp/ipykernel_20207/4253731750.py:22: RuntimeWarning: coroutine 'Page.wait_for_timeout' was never awaited
  page.wait_for_timeout(5000)


===== playwright page content part ======
[youtube] Extracting URL: https://www.youtube.com/embed/W3pS_HEbM_o
[youtube] W3pS_HEbM_o: Downloading webpage
[youtube] W3pS_HEbM_o: Downloading ios player API JSON
[youtube] W3pS_HEbM_o: Downloading android player API JSON
[youtube] W3pS_HEbM_o: Downloading player 23604418
[youtube] W3pS_HEbM_o: Downloading m3u8 information
[info] W3pS_HEbM_o: Downloading 1 format(s): 22
[download] Destination: ADENR173-CU/Video/W3pS_HEbM_o.mp4
[download] 100% of    2.43MiB in 00:00:07 at 355.81KiB/s   
[youtube] Extracting URL: https://www.youtube.com/embed/m6JaUBjl5mc
[youtube] m6JaUBjl5mc: Downloading webpage
[youtube] m6JaUBjl5mc: Downloading ios player API JSON
[youtube] m6JaUBjl5mc: Downloading android player API JSON
[youtube] m6JaUBjl5mc: Downloading m3u8 information
[info] m6JaUBjl5mc: Downloading 1 format(s): 22
[download] Destination: ADENR173-CU/Video/m6JaUBjl5mc.mp4
[download] 100% of    2.06MiB in 00:00:04 at 479.53KiB/s   
[youtube] Extracting 

In [6]:
await browser.close()
await playwright.stop()

In [7]:
# end time
end = time.time()

# total time taken
print("Execution time of the program is- ", end-start)

Execution time of the program is-  254.87275314331055
